In [19]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
import optuna

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import warnings
warnings.filterwarnings('ignore')

# Configuration

In [20]:
ROOT = ".."
INPUT = "input"
LISH_MOA = "lish-moa"
NUM_FOLD = 5
NUM_OPTUNA_TRIAL = 30
N_COMP_GENES = 50
N_COMP_CELLS = 15

# Read data

In [21]:
train = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "train_features.csv"))
test = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "test_features.csv"))
train_targets_scored = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "train_targets_scored.csv"))
train_targets_nonscored = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "train_targets_nonscored.csv"))
sub = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "sample_submission.csv"))

In [22]:
train_targets_scored.columns

Index(['sig_id', '5-alpha_reductase_inhibitor', '11-beta-hsd1_inhibitor',
       'acat_inhibitor', 'acetylcholine_receptor_agonist',
       'acetylcholine_receptor_antagonist', 'acetylcholinesterase_inhibitor',
       'adenosine_receptor_agonist', 'adenosine_receptor_antagonist',
       'adenylyl_cyclase_activator',
       ...
       'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist',
       'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor',
       'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b',
       'vitamin_d_receptor_agonist', 'wnt_inhibitor'],
      dtype='object', length=207)

In [23]:
for col in train_targets_scored.columns:
    if col != 'sig_id':
        c = train_targets_scored[col].value_counts()
        if c[1] <= NUM_FOLD:
            print(col)

atp-sensitive_potassium_channel_antagonist
erbb2_inhibitor


## train_features.csv

In [24]:
print(train.shape)

(23814, 876)


In [25]:
train.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


## train_targets_socred.csv

In [26]:
print(train_targets_scored.shape)

(23814, 207)


In [27]:
train_targets_scored.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## test_features.csv

In [28]:
print(test.shape)

(3982, 876)


In [29]:
test.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380


## train_targets_nonscored.csv (not use)

In [30]:
print(train_targets_nonscored.shape)

(23814, 403)


In [31]:
train_targets_nonscored.head()

,sig_id,abc_transporter_expression_enhancer,abl_inhibitor,ace_inhibitor,acetylcholine_release_enhancer,adenosine_deaminase_inhibitor,adenosine_kinase_inhibitor,adenylyl_cyclase_inhibitor,age_inhibitor,alcohol_dehydrogenase_inhibitor,...,ve-cadherin_antagonist,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## submission.csv

In [32]:
sub.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,id_001897cda,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2,id_002429b5b,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3,id_00276f245,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,id_0027f1083,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


# Preprocessing

## PCA features + Existing features

In [33]:
def make_pca_features(df_train:pd.DataFrame, df_test:pd.DataFrame, n_components:int, use_cols:list, gene_or_cell:str, concat_flg:bool):
    data = pd.concat([pd.DataFrame(df_train[use_cols]), pd.DataFrame(df_test[use_cols])])
    data_pca = PCA(n_components=n_components, random_state=334).fit_transform(data[use_cols])

    train_pca = data_pca[:df_train.shape[0]]
    test_pca = data_pca[-df_test.shape[0]:]

    train_pca = pd.DataFrame(train_pca, columns=['pca_'+gene_or_cell+str(i) for i in range(n_components)])
    test_pca = pd.DataFrame(test_pca, columns=['pca_'+gene_or_cell+str(i) for i in range(n_components)])

    if concat_flg:
        ret_df_train = pd.concat([df_train, train_pca], axis=1)
        ret_df_test = pd.concat([df_test, test_pca], axis=1)
    else:
        ret_df_train = pd.concat([df_train['sig_id'], train_pca], axis=1)
        ret_df_test = pd.concat([df_test['sig_id'], test_pca], axis=1)
    return ret_df_train, ret_df_test

In [34]:
GENES = [col for col in train.columns if col.startswith('g-')]
CELLS = [col for col in train.columns if col.startswith('c-')]

In [35]:
train, test = make_pca_features(train, test, N_COMP_GENES, GENES, 'G', True)
train, test = make_pca_features(train, test, N_COMP_CELLS, GENES, 'C', True)

# feature Selection using Variance Encoding

In [36]:
from sklearn.feature_selection import VarianceThreshold

var_thresh = VarianceThreshold(threshold=0.9)
data = train.append(test)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train.shape[0]]
test_features_transformed = data_transformed[-test.shape[0] : ]

train = pd.DataFrame(train[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])
train = pd.concat([train, pd.DataFrame(train_features_transformed)], axis=1)

test = pd.DataFrame(test[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])
test = pd.concat([test, pd.DataFrame(test_features_transformed)], axis=1)

In [37]:
train = train.merge(train_targets_scored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test[test['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]

In [ ]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

## Label Encoding

In [38]:
def label_encoding(train: pd.DataFrame, test: pd.DataFrame, encode_cols):
    n_train = len(train)
    train = pd.concat([train, test], sort=False).reset_index(drop=True)
    for f in encode_cols:
        try:
            lbl = preprocessing.LabelEncoder()
            train[f] = lbl.fit_transform(list(train[f].values))
        except:
            print(f)
    test = train[n_train:].reset_index(drop=True)
    train = train[:n_train]
    return train, test

In [39]:
train.select_dtypes(include=['object']).columns

Index(['sig_id', 'cp_type', 'cp_time', 'cp_dose'], dtype='object')

In [40]:
# train['cp_type'] = train['cp_type'].astype(str)
# train['cp_dose'] = train['cp_dose'].astype(str)
# test['cp_type'] = test['cp_type'].astype(str)
# test['cp_dose'] = test['cp_dose'].astype(str)

In [41]:
# train, test = label_encoding(train, test, ['cp_type', 'cp_dose'])

## Standrization

In [42]:
tr_mean = train.iloc[:, 1:].mean()
tr_std = train.iloc[:, 1:].std()
train.iloc[:, 1:] = (train.iloc[:, 1:] - tr_mean) / tr_std
test.iloc[:, 1:] = (test.iloc[:, 1:] - tr_mean) / tr_std

In [43]:
train.head()

,sig_id,cp_type,cp_time,cp_dose,0,1,2,3,4,5,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,NaN,-1.23765,-0.979949,0.549586,-0.401905,-0.735057,-0.273013,-0.734081,0.151063,...,-0.016536,-0.033768,-0.046815,-0.120861,-0.057767,-0.016536,-0.08835,-0.034438,-0.04219,-0.036996
1,id_000779bfc,NaN,1.23608,-0.979949,-0.142507,0.126192,-0.021845,0.912762,0.561225,0.476355,...,-0.016536,-0.033768,-0.046815,-0.120861,-0.057767,-0.016536,-0.08835,-0.034438,-0.04219,-0.036996
2,id_000a6266a,NaN,-0.000788961,-0.979949,0.245477,1.337724,-0.165074,-0.114604,1.168270,0.369302,...,-0.016536,-0.033768,-0.046815,-0.120861,-0.057767,-0.016536,-0.08835,-0.034438,-0.04219,-0.036996
3,id_0015fd391,NaN,-0.000788961,-0.979949,-0.554596,-0.418993,0.468566,3.886483,-0.562946,0.337371,...,-0.016536,-0.033768,-0.046815,-0.120861,-0.057767,-0.016536,-0.08835,-0.034438,-0.04219,-0.036996
4,id_001626bd3,NaN,1.23608,1.020415,-0.422582,0.773906,0.639331,1.302678,-0.575538,0.047968,...,-0.016536,-0.033768,-0.046815,-0.120861,-0.057767,-0.016536,-0.08835,-0.034438,-0.04219,-0.036996


In [44]:
test.head()

,sig_id,cp_type,cp_time,cp_dose,0,1,2,3,4,5,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,NaN,-1.23765,-0.979949,-0.577018,-0.658326,0.376431,1.431672,-0.017763,0.375108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id_001897cda,NaN,1.23608,-0.979949,-0.322730,1.003681,-0.558534,-0.439925,-0.159319,0.533532,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,id_00276f245,NaN,-1.23765,1.020415,0.143733,0.206709,0.359260,-0.655209,-0.894653,0.021491,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,id_0027f1083,NaN,-0.000788961,-0.979949,-0.473383,1.684317,0.130283,-0.656089,0.107145,0.739632,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,id_006fc47b8,NaN,-0.000788961,1.020415,0.061750,-0.828533,-1.788541,0.428933,-0.202674,1.065540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Cross Validation by using Optuna

In [45]:
def logistic_elasticnet_cv(data, targets, target_col, cv):
    def objective(trial):
        param = {
            'penalty': 'l1',
            'C': trial.suggest_loguniform('C', 2**(-10), 2**5),
            'solver': 'liblinear',
            'n_jobs': 8
        }
        
        df_x = data.drop(["sig_id"], axis=1)
        df_y = targets[target_col]
        score_logloss = 0.
        
        # cross validation
        for fold_id, (train_index, valid_index) in enumerate(tqdm(cv.split(df_x, df_y))):
            X_train = df_x.loc[train_index, :]
            y_train = df_y[train_index]
            X_valid = df_x.loc[valid_index, :]
            y_valid = df_y.loc[valid_index]
            
            model = LogisticRegression(**param)
            model.fit(X_train, y_train)
            y_pred = model.predict_proba(X_valid)
            score_logloss += log_loss(y_valid, y_pred, labels=[0, 1])

        score_logloss /= NUM_FOLD
        return score_logloss
    return objective
        

In [46]:
cv = StratifiedKFold(n_splits=NUM_FOLD, shuffle=True, random_state=0)

In [47]:
skip_cols = ["atp-sensitive_potassium_channel_antagonist", "erbb2_inhibitor"]

In [48]:
for target_col in tqdm(train_targets_scored.columns):
    if (target_col != "sig_id") and not (target_col in skip_cols):
        print('##################### CV START: {0} #####################'.format(target_col))
        
        # optimize by optuna
        study = optuna.create_study()
        study.optimize(logistic_elasticnet_cv(train, train_targets_scored, target_col, cv), n_trials=NUM_OPTUNA_TRIAL)
        
        # save optuna log
        df_trial = study.trials_dataframe()
        df_trial.to_pickle(os.path.join('report', 'LR_PCA_VarTh_OptunaLog_'+target_col+'.pkl'))

        # save best params found by CV
        bestparam_filename = 'LR_PCA_VarTh_BestParamsSelectedByCV_'+target_col+'.pkl'
        with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'wb') as f:
            pickle.dump(study.best_params, f)


  0%|                                                                                          | 0/207 [00:00<?, ?it/s]

##################### CV START: 5-alpha_reductase_inhibitor #####################



0it [00:00, ?it/s]
[W 2020-11-06 00:40:18,921] Setting status of trial#0 as TrialState.FAIL because of the following error: ValueError('Found input variables with inconsistent numbers of samples: [21948, 23814]')
Traceback (most recent call last):
  File "c:\users\owner\appdata\local\programs\python\python38\lib\site-packages\optuna\study.py", line 734, in _run_trial
    result = func(trial)
  File "<ipython-input-45-ecbc648f7ac0>", line 15, in objective
    for fold_id, (train_index, valid_index) in enumerate(tqdm(cv.split(df_x, df_y))):
  File "c:\users\owner\appdata\local\programs\python\python38\lib\site-packages\tqdm\std.py", line 1129, in __iter__
    for obj in iterable:
  File "c:\users\owner\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py", line 328, in split
    X, y, groups = indexable(X, y, groups)
  File "c:\users\owner\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py", line 293, in indexable
 

ValueError: Found input variables with inconsistent numbers of samples: [21948, 23814]

# Train & Predict by best model

In [ ]:
submission = sub.copy()

In [ ]:
def logistic_elasticnet_bestparam(data_train, targets_train, data_test, target_col, best_params, submission):
    print('##################### TraingByBestParam START: {0} #####################'.format(target_col))
    
    # prepare data set
    X_train = data_train.drop(["sig_id"], axis=1)
    y_train = targets_train[target_col]
    X_test = data_test.drop(["sig_id"], axis=1)
    
    # train XGBoost by using best_params
    model = LogisticRegression(**best_params)
    model.fit(X_train, y_train)
    
    # predict for test
    submission[target_col] = model.predict_proba(X_test)[:, 1]
    
    # save model for test
    modelfile = 'LR_PCA_VarTh_BestModel4test_'+target_col+'.pkl'
    with open(os.path.join('result', 'best_model', modelfile), 'wb') as f:
        pickle.dump(model, f)
    

In [ ]:
# for target_col in tqdm(train_targets_scored.columns):
for target_col in tqdm(train_targets_scored.columns):
    if target_col != "sig_id" and not (target_col in skip_cols):
        # read best param selected by CV
        bestparam_filename = 'LR_PCA_VarTh_BestParamsSelectedByCV_'+target_col+'.pkl'
        with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'rb') as f:
            best_params = pickle.load(f)
            
        best_params['penalty'] = 'l1'
        best_params['n_jobs'] = 8
        best_params['solver'] = 'liblinear'
        
        logistic_elasticnet_bestparam(train, train_targets_scored, test, target_col, best_params, submission)
    elif target_col in skip_cols:
        submission[target_col] = 1e-05

In [ ]:
submission.to_csv('submission.csv', index=False)